In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
pd.set_option("display.max_columns",75)

In [2]:
## FEATURE FUNCS

def strip_and_make_0_float(x):

    return x.str.split(expand=True)[0].astype(float)

def featurize(table,process_name=True):

    table.columns = [
        'rk','team','conf','g','rec','adjoe','adjde','barthag','efg_pct','efgd_pct',
        'tor','tord','orb','drb','ftr','ftrd','2p_pct','2pd_pct','3p_pct','3pd_pct','adj_t','wab'
    ]

    table = table[table['rk']!='Rk'].copy()
    table[['w','l']]=table['rec'].str.split(expand=True)[0].str.split("–",expand=True).astype(int)
    table['win_perc'] = table['w']/(table['w']+table['l'])

    if process_name:
        table[['team','tourney']]=table['team'].str.split(",",expand=True)
        table['team'] = table['team'].str.replace(" seed","").apply(
            lambda x: ''.join([i for i in x if (i.isalpha())|(i==" ")])
        ).str.strip()
        table['tourney'] = table['tourney'].fillna('no tourney').str.strip()

        tourney_values = {
            'no tourney':128,
            'R68':68,
            'R64':64,
            'R32':32,
            'Sweet Sixteen':16,
            'Elite Eight':8,
            'Final Four':4,
            'Finals':2,
            'CHAMPS':1,
        }

        table['OUTCOME'] = table['tourney'].map(tourney_values)

    table['adjoe'] = strip_and_make_0_float(table['adjoe'])
    table['adjde'] = strip_and_make_0_float(table['adjde'])
    table['barthag'] = strip_and_make_0_float(table['barthag'])
    table['efg_pct'] = strip_and_make_0_float(table['efg_pct'])
    table['efgd_pct'] = strip_and_make_0_float(table['efgd_pct'])
    table['tor'] = strip_and_make_0_float(table['tor'])
    table['tord'] = strip_and_make_0_float(table['tord'])
    table['orb'] = strip_and_make_0_float(table['orb'])
    table['drb'] = strip_and_make_0_float(table['drb'])
    table['ftr'] = strip_and_make_0_float(table['ftr'])
    table['ftrd'] = strip_and_make_0_float(table['ftrd'])
    table['2p_pct'] = strip_and_make_0_float(table['2p_pct'])
    table['2pd_pct'] = strip_and_make_0_float(table['2pd_pct'])
    table['3p_pct'] = strip_and_make_0_float(table['3p_pct'])
    table['3pd_pct'] = strip_and_make_0_float(table['3pd_pct'])
    table['adj_t'] = strip_and_make_0_float(table['adj_t'])
    table['wab'] = strip_and_make_0_float(table['wab'])


    conferences = ['WCC', 'Amer', 'B12', 'ACC', 'SEC', 'BE', 'P12', 'B10', 'MWC',
        'MVC', 'A10', 'OVC', 'CUSA', 'AE', 'SC', 'WAC', 'Sum', 'CAA',
        'MAAC', 'MAC', 'Ivy', 'ASun', 'Pat', 'SB', 'BW', 'BSth', 'BSky',
        'NEC', 'Horz', 'SWAC', 'MEAC', 'Slnd']


    for c in conferences:
        table[c] = np.where(table['conf']==c,1,0)

    return table

In [ ]:
years = [
    2008,
    2009,
    2010,
    2011,
    2012,
    2013,
    2014,
    2015,
    2016,
    2017,
    2018,
    2019,
    2021,
    2022
         ]

out = pd.DataFrame()
for year in years:
    print(year)

    url = f"https://barttorvik.com/trank.php?year={year}&type=R"

    tables = pd.read_html(url)

    table = tables[0]

    tmp = featurize(table)

    tmp['year']=year

    out = pd.concat([out,tmp])

In [ ]:
out

In [ ]:
training_mask = out['year'].isin([2008,
    2009,
    2010,
    2011,
    2012,
    2013,
    2014,
    2015,
    2016,
    2017,
    2018,
    2019,
    2021,])
validation_mask = out['year'].isin([2022])


In [ ]:
features = ['adjoe', 'adjde', 'barthag',
       'efg_pct', 'efgd_pct', 'tor', 'tord', 'orb', 'drb', 'ftr', 'ftrd',
       '2p_pct', '2pd_pct', '3p_pct', '3pd_pct',
       'win_perc', 
       'WCC', 'Amer', 'B12', 'ACC', 'SEC',
       'BE', 'P12', 'B10', 'MWC', 'MVC', 'A10', 'OVC', 'CUSA', 'AE', 'SC',
       'WAC', 'Sum', 'CAA', 'MAAC', 'MAC', 'Ivy', 'ASun', 'Pat', 'SB', 'BW',
       'BSth', 'BSky', 'NEC', 'Horz', 'SWAC', 'MEAC', 'Slnd'
       ]
target = 'OUTCOME'

In [ ]:
X = out[training_mask][features].copy()
y = out[training_mask][target].copy()

In [ ]:
model = Pipeline(
    steps=[
    ('scaler',MinMaxScaler()),
    ('learner',RandomForestRegressor(n_estimators=500,random_state=50,#max_depth=8
                                     ))
    ]
)

In [ ]:
model.fit(X,y)

In [ ]:
yfit = model.predict(X)

In [ ]:
Xx = out[validation_mask][features].copy()
yy = out[validation_mask][target].copy()
yval = model.predict(Xx)

In [ ]:
val_result = pd.DataFrame({
    'team':out[validation_mask]['team'],
    'OUTCOME':yy})
val_result['PREDICTION_NUMERIC'] = yval
val_result['PREDICTION_RANK']=val_result['PREDICTION_NUMERIC'].rank(ascending=True,)

In [ ]:
val_result.sort_values(by='PREDICTION_RANK',ascending=True).head(50)

In [ ]:
testyear = 2023

url = f"https://barttorvik.com/trank.php?year={testyear}&type=R"

tables = pd.read_html(url)

table = tables[0]

tmp = featurize(table,process_name=False)

tmp['year']=year


In [ ]:
prediction_df = pd.DataFrame({
    'team':tmp['team'],
    'rating':model.predict(tmp[features])
})
    
prediction_df

In [ ]:
prediction_df.sort_values(by='rating',ascending=True).head(50)

In [ ]:
url_root = "https://barttorvik.com/trank.php?year={y}&type=R"


In [ ]:
url_root.format(y=2021)